In [57]:
include("postgresql.jl")
include("TunnelingModule.jl")

##get training data
#8 combinations of handedness and ball type
fast_ll_train = get_data(true, islhp=true, islhb=true)
fast_lr_train = get_data(true, islhp=true, islhb=false)
fast_rl_train = get_data(true, islhp=false, islhb=true)
fast_rr_train = get_data(true, islhp=false, islhb=false)
slow_ll_train = get_data(true, islhp=true, islhb=true, fast=false)
slow_lr_train = get_data(true, islhp=true, islhb=false, fast=false)
slow_rl_train = get_data(true, islhp=false, islhb=true, fast=false)
slow_rr_train = get_data(true, islhp=false, islhb=false, fast=false)

##get test data
fast_ll_test = get_data(false, islhp=true, islhb=true)
fast_lr_test = get_data(false, islhp=true, islhb=false)
fast_rl_test = get_data(false, islhp=false, islhb=true)
fast_rr_test = get_data(false, islhp=false, islhb=false)
slow_ll_test = get_data(false, islhp=true, islhb=true, fast=false)
slow_lr_test = get_data(false, islhp=true, islhb=false, fast=false)
slow_rl_test = get_data(false, islhp=false, islhb=true, fast=false)
slow_rr_test = get_data(false, islhp=false, islhb=false, fast=false)

##calculate distances and ratios
fast_ll_train = calc_ratios(fast_ll_train)
fast_lr_train = calc_ratios(fast_lr_train)
fast_rl_train = calc_ratios(fast_rl_train)
fast_rr_train = calc_ratios(fast_rr_train)
slow_ll_train = calc_ratios(slow_ll_train)
slow_lr_train = calc_ratios(slow_lr_train)
slow_rl_train = calc_ratios(slow_rl_train)
slow_rr_train = calc_ratios(slow_rr_train)

##prep test data
fast_ll_test = calc_ratios(fast_ll_test)
fast_lr_test = calc_ratios(fast_lr_test)
fast_rl_test = calc_ratios(fast_rl_test)
fast_rr_test = calc_ratios(fast_rr_test)
slow_ll_test = calc_ratios(slow_ll_test)
slow_lr_test = calc_ratios(slow_lr_test)
slow_rl_test = calc_ratios(slow_rl_test)
slow_rr_test = calc_ratios(slow_rr_test)

Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pit

([5.67331 3.00488; 1.71009 0.741705; … ; 1.15943 0.283392; 2.25952 0.774491], [0.0376189, -0.0512863, -0.183523, -0.0599418, -0.0512863, 0.511992, -0.0629139, 0.0, 0.0410767, -0.145117  …  -0.0655811, -0.223813, 0.474374, -0.0512863, 0.0470651, -0.223813, 0.067862, 0.0470651, 0.0471762, -0.0599418])

We preprocess the data to be a tuple (X,Y) with X a matrix and Y a vector of runValues.<br>
The matrix will have two colums, the first being the ratio of release distance to tunnel distance and the second distance at the base to tunnel distance. Distances are calculated with respect to the mean release/tunnel/break point (in the x-z plane).

The dataset for fast balls and sinkers with both right-handed pitchers and batters is the largest set, so we will use this for timing of the algorithms.

In [64]:
##linear regression
@time linReg_fast_ll = lin_reg_pred(fast_ll_train[1],fast_ll_train[2],fast_ll_test[1])
linReg_fast_lr = lin_reg_pred(fast_lr_train[1],fast_lr_train[2],fast_lr_test[1])
linReg_fast_rl = lin_reg_pred(fast_rl_train[1],fast_rl_train[2],fast_rl_test[1])
@time linReg_fast_rr = lin_reg_pred(fast_rr_train[1],fast_rr_train[2],fast_rr_test[1])
linReg_slow_ll = lin_reg_pred(slow_ll_train[1],slow_ll_train[2],slow_ll_test[1])
linReg_slow_lr = lin_reg_pred(slow_lr_train[1],slow_lr_train[2],slow_lr_test[1])
linReg_slow_rl = lin_reg_pred(slow_rl_train[1],slow_rl_train[2],slow_rl_test[1])
linReg_slow_rr = lin_reg_pred(slow_rr_train[1],slow_rr_train[2],slow_rr_test[1])

##neural net
@time m_fast_ll = nn_full(fast_ll_train[1],fast_ll_train[2])
m_fast_lr = nn_full(fast_lr_train[1],fast_lr_train[2])
m_fast_rl = nn_full(fast_rl_train[1],fast_rl_train[2])
@time m_fast_rr = nn_full(fast_rr_train[1],fast_rr_train[2])
m_slow_ll = nn_full(slow_ll_train[1],slow_ll_train[2])
m_slow_lr = nn_full(slow_lr_train[1],slow_lr_train[2])
m_slow_rl = nn_full(slow_rl_train[1],slow_rl_train[2])
m_slow_rr = nn_full(slow_rr_train[1],slow_rr_train[2])

#save parameters of nets
#save("nn_params.jld", "m_fast_ll", params(m_fast_ll), "m_fast_lr", params(m_fast_lr), "m_fast_rl", params(m_fast_rl), "m_fast_rr", params(m_fast_rr), "m_slow_ll", params(m_slow_ll), "m_slow_lr", params(m_slow_lr), "m_slow_rl", params(m_slow_rl), "m_slow_rr", params(m_slow_rr))
 
##Nearest neighbor classification
#k=16 seemed to give minimum mse in our tests
@time knn_fast_ll = knn_pred(fast_ll_train[1],fast_ll_train[2],fast_ll_test[1],16)
knn_fast_lr = knn_pred(fast_lr_train[1],fast_lr_train[2],fast_lr_test[1],16)
knn_fast_rl = knn_pred(fast_rl_train[1],fast_rl_train[2],fast_rl_test[1],16)
@time knn_fast_rr = knn_pred(fast_rr_train[1],fast_rr_train[2],fast_rr_test[1],16)
knn_slow_ll = knn_pred(slow_ll_train[1],slow_ll_train[2],slow_ll_test[1],16)
knn_slow_lr = knn_pred(slow_lr_train[1],slow_lr_train[2],slow_lr_test[1],16)
knn_slow_rl = knn_pred(slow_rl_train[1],slow_rl_train[2],slow_rl_test[1],16)
knn_slow_rr = knn_pred(slow_rr_train[1],slow_rr_train[2],slow_rr_test[1],16)

2273.289453 seconds (620.56 k allocations: 269.358 GiB, 1.96% gc time)


60882×1 Array{Float64,2}:
 -0.21046511086254283  
  0.12600369423770993  
  0.071834285201982    
 -0.08820426679986743  
  0.011177167007780694 
 -0.07173453897331472  
  0.06990323477830879  
 -0.0635713001226146   
  0.12701014607088942  
  0.042712184206237444 
 -0.0008928640595879174
  0.03712047482388185  
 -0.042597852923827915 
  ⋮                    
  0.01228197221701135  
 -0.0025064841659371262
 -0.052781045886313806 
  0.045539285542270574 
 -0.06833988287474625  
 -0.05938198814730592  
 -0.08620631773107391  
  0.11873536289029556  
  0.009819817095831518 
 -0.1533262966732252   
  0.06441411655249855  
 -0.062468475106695606 

In [68]:
##read in model parameters
mpar_fast_ll = load("nn_params.jld", "m_fast_ll")
mpar_fast_lr = load("nn_params.jld", "m_fast_lr")
mpar_fast_rl = load("nn_params.jld", "m_fast_rl")
mpar_fast_rr = load("nn_params.jld", "m_fast_rr")
mpar_slow_ll = load("nn_params.jld", "m_slow_ll")
mpar_slow_lr = load("nn_params.jld", "m_slow_lr")
mpar_slow_rl = load("nn_params.jld", "m_slow_rl")
mpar_slow_rr = load("nn_params.jld", "m_slow_rr")

##neural net predictions
#from model parameters
@time m_fast_ll = nn_pred_params(mpar_fast_ll, fast_ll_test[1])
m_fast_lr = nn_pred_params(mpar_fast_lr, fast_lr_test[1])
m_fast_rl = nn_pred_params(mpar_fast_rl, fast_rl_test[1])
@time m_fast_rr = nn_pred_params(mpar_fast_rr, fast_rr_test[1])
m_slow_ll = nn_pred_params(mpar_slow_ll, slow_ll_test[1])
m_slow_lr = nn_pred_params(mpar_slow_lr, slow_lr_test[1])
m_slow_rl = nn_pred_params(mpar_slow_rl, slow_rl_test[1])
m_slow_rr = nn_pred_params(mpar_slow_rr, slow_rr_test[1])

#or directly from model
#@time m_fast_ll = nn_pred_orig(m_fast_ll, fast_ll_test[1])
#m_fast_lr = nn_pred_orig(m_fast_lr, fast_lr_test[1])
#m_fast_rl = nn_pred_orig(m_fast_rl, fast_rl_test[1])
#@time m_fast_rr = nn_pred_orig(m_fast_rr, fast_rr_test[1])
#m_slow_ll = nn_pred_orig(m_slow_ll, slow_ll_test[1])
#m_slow_lr = nn_pred_orig(m_slow_lr, slow_lr_test[1])
#m_slow_rl = nn_pred_orig(m_slow_rl, slow_rl_test[1])
#m_slow_rr = nn_pred_orig(m_slow_rr, slow_rr_test[1])

  0.026401 seconds (77 allocations: 20.124 MiB, 24.60% gc time)


Tracked 60882×1 Adjoint{Float64,Array{Float64,2}}:
 -0.02029604352826897  
  0.008439756337136997 
 -0.0540904197551439   
 -0.005121696972588834 
  0.0069186224958420715
 -0.07374624139611338  
  0.013414002618059112 
  0.0010323181371703011
  0.005324141515265366 
  0.0069869070741119385
  0.011186780354057558 
 -0.0006266585241679425
  0.004267619996770394 
  ⋮                    
 -0.014180798690656036 
 -0.006692068797921115 
 -0.006784482372658593 
 -0.026593797678204238 
  0.002941086762755047 
 -0.024338313746816317 
  0.007391467660345466 
 -0.0346134560578846   
 -0.004626021466620391 
 -0.00433704995821421  
  0.013291736513837617 
  0.010440461538112455 

In [72]:
include("TunnelingModule.jl")
##calculate metrics
sets = ["Fastball/sinkers - LL", "Fastball/sinkers - LR", "Fastball/sinkers - RL", "Fastball/sinkers - RR", "Other pitches - LL", "Other pitches - LR", "Other pitches - RL", "Other pitches - RR"]
mses_lR = [
    mse(linReg_fast_ll, fast_ll_test[2]),
    mse(linReg_fast_lr, fast_lr_test[2]),
    mse(linReg_fast_rl, fast_rl_test[2]),
    mse(linReg_fast_rr, fast_rr_test[2]),
    mse(linReg_slow_ll, slow_ll_test[2]),
    mse(linReg_slow_lr, slow_lr_test[2]),
    mse(linReg_slow_rl, slow_rl_test[2]),
    mse(linReg_slow_rr, slow_rr_test[2])
]
acc_decs_lR = [
    calc_acc_dec(linReg_fast_ll, fast_ll_test[2]),
    calc_acc_dec(linReg_fast_lr, fast_lr_test[2]),
    calc_acc_dec(linReg_fast_rl, fast_rl_test[2]),
    calc_acc_dec(linReg_fast_rr, fast_rr_test[2]),
    calc_acc_dec(linReg_slow_ll, slow_ll_test[2]),
    calc_acc_dec(linReg_slow_lr, slow_lr_test[2]),
    calc_acc_dec(linReg_slow_rl, slow_rl_test[2]),
    calc_acc_dec(linReg_slow_rr, slow_rr_test[2])
]
acc_ints_lR = [
    calc_acc_int(linReg_fast_ll, fast_ll_test[2]),
    calc_acc_int(linReg_fast_lr, fast_lr_test[2]),
    calc_acc_int(linReg_fast_rl, fast_rl_test[2]),
    calc_acc_int(linReg_fast_rr, fast_rr_test[2]),
    calc_acc_int(linReg_slow_ll, slow_ll_test[2]),
    calc_acc_int(linReg_slow_lr, slow_lr_test[2]),
    calc_acc_int(linReg_slow_rl, slow_rl_test[2]),
    calc_acc_int(linReg_slow_rr, slow_rr_test[2])
]
mses_m = [
    mse(m_fast_ll, fast_ll_test[2]),
    mse(m_fast_lr, fast_lr_test[2]),
    mse(m_fast_rl, fast_rl_test[2]),
    mse(m_fast_rr, fast_rr_test[2]),
    mse(m_slow_ll, slow_ll_test[2]),
    mse(m_slow_lr, slow_lr_test[2]),
    mse(m_slow_rl, slow_rl_test[2]),
    mse(m_slow_rr, slow_rr_test[2])
]
acc_decs_m = [
    calc_acc_dec(m_fast_ll, fast_ll_test[2]),
    calc_acc_dec(m_fast_lr, fast_lr_test[2]),
    calc_acc_dec(m_fast_rl, fast_rl_test[2]),
    calc_acc_dec(m_fast_rr, fast_rr_test[2]),
    calc_acc_dec(m_slow_ll, slow_ll_test[2]),
    calc_acc_dec(m_slow_lr, slow_lr_test[2]),
    calc_acc_dec(m_slow_rl, slow_rl_test[2]),
    calc_acc_dec(m_slow_rr, slow_rr_test[2])
]
acc_ints_m = [
    calc_acc_int(m_fast_ll, fast_ll_test[2]),
    calc_acc_int(m_fast_lr, fast_lr_test[2]),
    calc_acc_int(m_fast_rl, fast_rl_test[2]),
    calc_acc_int(m_fast_rr, fast_rr_test[2]),
    calc_acc_int(m_slow_ll, slow_ll_test[2]),
    calc_acc_int(m_slow_lr, slow_lr_test[2]),
    calc_acc_int(m_slow_rl, slow_rl_test[2]),
    calc_acc_int(m_slow_rr, slow_rr_test[2])
]
mses_knn = [
    mse(knn_fast_ll, fast_ll_test[2]),
    mse(knn_fast_lr, fast_lr_test[2]),
    mse(knn_fast_rl, fast_rl_test[2]),
    mse(knn_fast_rr, fast_rr_test[2]),
    mse(knn_slow_ll, slow_ll_test[2]),
    mse(knn_slow_lr, slow_lr_test[2]),
    mse(knn_slow_rl, slow_rl_test[2]),
    mse(knn_slow_rr, slow_rr_test[2])
]
acc_decs_knn = [
    calc_acc_dec(knn_fast_ll, fast_ll_test[2]),
    calc_acc_dec(knn_fast_lr, fast_lr_test[2]),
    calc_acc_dec(knn_fast_rl, fast_rl_test[2]),
    calc_acc_dec(knn_fast_rr, fast_rr_test[2]),
    calc_acc_dec(knn_slow_ll, slow_ll_test[2]),
    calc_acc_dec(knn_slow_lr, slow_lr_test[2]),
    calc_acc_dec(knn_slow_rl, slow_rl_test[2]),
    calc_acc_dec(knn_slow_rr, slow_rr_test[2])
]
acc_ints_knn = [
    calc_acc_int(knn_fast_ll, fast_ll_test[2]),
    calc_acc_int(knn_fast_lr, fast_lr_test[2]),
    calc_acc_int(knn_fast_rl, fast_rl_test[2]),
    calc_acc_int(knn_fast_rr, fast_rr_test[2]),
    calc_acc_int(knn_slow_ll, slow_ll_test[2]),
    calc_acc_int(knn_slow_lr, slow_lr_test[2]),
    calc_acc_int(knn_slow_rl, slow_rl_test[2]),
    calc_acc_int(knn_slow_rr, slow_rr_test[2])
]
DataFrame(Dataset=sets, LinRegMSE=mses_lR, NeuralNetMSE=mses_m, knnMSE=mses_knn, LinRegAccuracyDecimal=acc_decs_lR, NeuralNetAccuracyDecimal=acc_decs_m, knnAccuracyDecimal=acc_decs_knn, LinRegAccuracyInteger=acc_ints_lR, NeuralNetAccuracyInteger=acc_ints_m, knnAccuracyInteger=acc_ints_knn)


,Dataset,LinRegMSE,NeuralNetMSE,knnMSE,LinRegAccuracyDecimal,NeuralNetAccuracyDecimal,knnAccuracyDecimal,LinRegAccuracyInteger,NeuralNetAccuracyInteger,knnAccuracyInteger
,String,Float64,Flux…,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Fastball/sinkers - LL,0.0422613,0.0422143 (tracked),0.0599847,29.4523,29.5058,28.4654,97.4602,97.4602,96.5499
2,Fastball/sinkers - LR,0.04827,0.0481913 (tracked),0.0704321,31.1462,31.1054,27.8087,97.1019,97.1019,95.7256
3,Fastball/sinkers - RL,0.0444328,0.0445178 (tracked),0.0673063,31.9407,32.3025,27.4115,97.2435,97.2435,95.7705
4,Fastball/sinkers - RR,0.0452079,0.0460184 (tracked),0.0655747,30.6617,32.3157,28.104,97.138,97.1419,96.0255
5,Other pitches - LL,0.034734,0.0347945 (tracked),0.0497866,36.1274,36.2427,31.9573,97.5168,97.5168,96.7414
6,Other pitches - LR,0.0438656,0.0439397 (tracked),0.0638549,35.1279,35.4101,29.1646,96.9611,96.9611,95.8058
7,Other pitches - RL,0.0429894,0.0619363 (tracked),0.0675072,34.5483,20.8749,28.5384,97.157,97.1548,95.7072
8,Other pitches - RR,0.0426293,0.0425988 (tracked),0.0627457,34.7853,34.9003,28.956,97.1272,97.1272,95.938
